# Analyzing the year-to-date border patrol encounter dataset

The dashboard is [here](https://www.cbp.gov/newsroom/stats/southwest-land-border-encounters). That page is hard to scrape ([link](https://stackoverflow.com/questions/79784978/importing-a-table-from-a-webpage-as-a-dataframe-in-python/79785744#79785744)). However, it links to where you can download the data as a csv that powers the dashboard ([link](https://www.cbp.gov/newsroom/stats/cbp-public-data-portal)). 

I downloaded the most recent data (Today is 2025-10-08 and the most recent data was until August). Hopefully all files are be in the same format 🤞.

Here is a notebook showing how to work with the files.

## Import and view data

In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import encounters

df = pd.read_csv("sbo-encounters-fy22-fy25-aug.csv")
df.head()

,Fiscal Year,Month Grouping,Month (abbv),Component,Demographic,Citizenship Grouping,Title of Authority,Encounter Type,Encounter Count
0,2022,FYTD,OCT,Office of Field Operations,Accompanied Minors,El Salvador,Title 42,Expulsions,1
1,2022,FYTD,OCT,Office of Field Operations,Accompanied Minors,Guatemala,Title 42,Expulsions,3
2,2022,FYTD,OCT,Office of Field Operations,Accompanied Minors,Honduras,Title 42,Expulsions,1
3,2022,FYTD,OCT,Office of Field Operations,Accompanied Minors,Mexico,Title 42,Expulsions,112
4,2022,FYTD,OCT,Office of Field Operations,Accompanied Minors,Mexico,Title 8,Inadmissibles,66


## Subset to FY2025 data

We already have all historic data from another file. So subset to the most recent year. Also, we are interested in USBP Encounters, and are ignoring OFO ones for this analysis. So make this dataset match the one we're merging with.

In [2]:
mask = (df["Fiscal Year"] == "2025 (FYTD)") & (df["Component"] == "U.S. Border Patrol")
df = df[mask]
df.head()

,Fiscal Year,Month Grouping,Month (abbv),Component,Demographic,Citizenship Grouping,Title of Authority,Encounter Type,Encounter Count
1613,2025 (FYTD),FYTD,APR,U.S. Border Patrol,FMUA,El Salvador,Title 8,Apprehensions,14
1614,2025 (FYTD),FYTD,APR,U.S. Border Patrol,FMUA,Guatemala,Title 8,Apprehensions,45
1615,2025 (FYTD),FYTD,APR,U.S. Border Patrol,FMUA,Honduras,Title 8,Apprehensions,33
1616,2025 (FYTD),FYTD,APR,U.S. Border Patrol,FMUA,Mexico,Title 8,Apprehensions,190
1617,2025 (FYTD),FYTD,APR,U.S. Border Patrol,FMUA,Other,Title 8,Apprehensions,252


## Convert FY2025 data to calendar year

The data uses a fiscal year, and we need it in a calendar year. This means converting dates like "FY2025 October" to "2024 October".


In [3]:
df["Year"] = df["Fiscal Year"].str.split().str[0]  # .str in Pandas is so weird.
df["fiscal_date"] = df["Year"] + " " + df["Month (abbv)"]
df["fiscal_date"] = pd.to_datetime(df.fiscal_date, format="%Y %b")  # Day defaults to 1
df["date"] = df.fiscal_date.apply(encounters.convert_fiscal_date_to_calendar_date)

df.sort_values("date")

,Fiscal Year,Month Grouping,Month (abbv),Component,Demographic,Citizenship Grouping,Title of Authority,Encounter Type,Encounter Count,Year,fiscal_date,date
1935,2025 (FYTD),FYTD,OCT,U.S. Border Patrol,UAC,Other,Title 8,Apprehensions,504,2025,2025-10-01,2024-10-01
1934,2025 (FYTD),FYTD,OCT,U.S. Border Patrol,UAC,Mexico,Title 8,Apprehensions,1985,2025,2025-10-01,2024-10-01
1921,2025 (FYTD),FYTD,OCT,U.S. Border Patrol,FMUA,El Salvador,Title 8,Apprehensions,756,2025,2025-10-01,2024-10-01
1922,2025 (FYTD),FYTD,OCT,U.S. Border Patrol,FMUA,Guatemala,Title 8,Apprehensions,1738,2025,2025-10-01,2024-10-01
1923,2025 (FYTD),FYTD,OCT,U.S. Border Patrol,FMUA,Honduras,Title 8,Apprehensions,1168,2025,2025-10-01,2024-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...
1650,2025 (FYTD),FYTD,AUG,U.S. Border Patrol,Single Adults,Honduras,Title 8,Apprehensions,333,2025,2025-08-01,2025-08-01
1651,2025 (FYTD),FYTD,AUG,U.S. Border Patrol,Single Adults,Mexico,Title 8,Apprehensions,3713,2025,2025-08-01,2025-08-01
1652,2025 (FYTD),FYTD,AUG,U.S. Border Patrol,Single Adults,Other,Title 8,Apprehensions,645,2025,2025-08-01,2025-08-01
1656,2025 (FYTD),FYTD,AUG,U.S. Border Patrol,UAC,Mexico,Title 8,Apprehensions,463,2025,2025-08-01,2025-08-01


### Sum values for each month

The data is monthly, but separated by things like the citizenship of people encountered. We just want one value for each month. We can then graph the result.

In [4]:
df = df.groupby("date")["Encounter Count"].sum().reset_index()
df.head()

,date,Encounter Count
0,2024-10-01,56520
1,2024-11-01,46612
2,2024-12-01,47324
3,2025-01-01,29107
4,2025-02-01,8353


In [6]:
fig = px.line(
    df,
    x="date",
    y="Encounter Count",
    title="Border Patrol Encounters at the Southwest Land Border<br><sup>Fiscal Year 2025 to date</sup>",
    labels={"date": "Date", "Encounter Count": "Encounters"},
    markers=True,
)

# Add a vertical line on the date the administration started, and write the presdient's name on the top
max_y = df["Encounter Count"].max()
trump_start = datetime(2025, 1, 20)

fig.add_vline(x=trump_start, line_color="black", line_dash="dash")
fig.add_annotation(
    x=trump_start,
    y=max_y,
    text="Donald Trump",
    xanchor="left",
    xshift=5,
    showarrow=False,
    yanchor="bottom",
)

fig.show()

# Now combine with historic data

In [7]:
df_historic = encounters.get_monthly_region_df()
df_historic.head()

,date,region,quantity
0,1999-10-01,Coastal Border,740
1,1999-10-01,Northern Land Border,1250
2,1999-10-01,Southwest Land Border,87820
3,1999-11-01,Coastal Border,500
4,1999-11-01,Northern Land Border,960


Get the new dataset in the same format as the historic one.

In [8]:
df_ytd = df.copy()
df_ytd["region"] = "Southwest Land Border"
df_ytd = df_ytd.rename(columns={"Encounter Count": "quantity"})
df_ytd = df_ytd[["date", "region", "quantity"]]
df_ytd.head()

,date,region,quantity
0,2024-10-01,Southwest Land Border,56520
1,2024-11-01,Southwest Land Border,46612
2,2024-12-01,Southwest Land Border,47324
3,2025-01-01,Southwest Land Border,29107
4,2025-02-01,Southwest Land Border,8353


In [9]:
df_all = pd.concat([df_historic, df_ytd], ignore_index=True)
df_all

,date,region,quantity
0,1999-10-01,Coastal Border,740
1,1999-10-01,Northern Land Border,1250
2,1999-10-01,Southwest Land Border,87820
3,1999-11-01,Coastal Border,500
4,1999-11-01,Northern Land Border,960
...,...,...,...
912,2025-04-01,Southwest Land Border,8377
913,2025-05-01,Southwest Land Border,8725
914,2025-06-01,Southwest Land Border,6068
915,2025-07-01,Southwest Land Border,4596


We're focused on just the Southwest Land Border here, so subset again.

In [10]:
mask = df_all["region"] == "Southwest Land Border"
df_all = df_all[mask]
df_all

,date,region,quantity
2,1999-10-01,Southwest Land Border,87820
5,1999-11-01,Southwest Land Border,74360
8,1999-12-01,Southwest Land Border,66260
11,2000-01-01,Southwest Land Border,175380
14,2000-02-01,Southwest Land Border,200620
...,...,...,...
912,2025-04-01,Southwest Land Border,8377
913,2025-05-01,Southwest Land Border,8725
914,2025-06-01,Southwest Land Border,6068
915,2025-07-01,Southwest Land Border,4596


# TODO: Note that there are duplicates!

Looks like 2024-10-01 and 2024-11-01 are duplicates, but have the same value. It looks like they publish the annual data in January. And so perhaps as a convenience they include values for October and November (the first two months of the new fiscal year). I should add in code that:

1. for all duplicate (dates), assert other values are identical
2. remove duplicates.

In [ ]:
df_all[df_all["date"] >= datetime(year=2024, month=1, day=1)]

,date,region,quantity
875,2024-01-01,Southwest Land Border,124220
878,2024-02-01,Southwest Land Border,140640
881,2024-03-01,Southwest Land Border,137470
884,2024-04-01,Southwest Land Border,128900
887,2024-05-01,Southwest Land Border,117910
890,2024-06-01,Southwest Land Border,83530
893,2024-07-01,Southwest Land Border,56400
896,2024-08-01,Southwest Land Border,58010
899,2024-09-01,Southwest Land Border,53860
902,2024-10-01,Southwest Land Border,56530


## Graph the merged dataset

In [12]:
fig = px.line(
    df_all,
    x="date",
    y="quantity",
    title="Border Patrol Encounters at the Southwest Land Border",
    labels={"quantity": "Encounters", "date": "Date"},
)

administrations = [
    # Include Clinton for reference, but comment out bc his administration did not start during the data period
    # {"President": "Bill Clinton", "Start": datetime(1993, 1, 20)},
    {"President": "George W. Bush", "Start": datetime(2001, 1, 20)},
    {"President": "Barack Obama", "Start": datetime(2009, 1, 20)},
    {"President": "Donald Trump", "Start": datetime(2017, 1, 20)},
    {"President": "Joe Biden", "Start": datetime(2021, 1, 20)},
    {
        "President": "",
        "Start": datetime(2025, 1, 20),
    },  # Not enough space to include a name
]

# Add a vertical line on the date the administration started, and write the presdient's name on the top
max_y = df_all["quantity"].max()

for one_administration in administrations:
    fig.add_vline(x=one_administration["Start"], line_color="black", line_dash="dash")
    fig.add_annotation(
        x=one_administration["Start"],
        y=max_y,
        text=one_administration["President"],
        xanchor="left",
        xshift=5,
        showarrow=False,
        yanchor="bottom",
    )

fig.show()